In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "STATUS",
                "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "ASK_AMT", "IS_SUCCESSFUL"]].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
print(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1500
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# Add first hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim= num_features))

# Add second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Add third hidden layer (attempt 1 at optimization by adding another hidden layer)
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Add fourth hidden layer (attempt 2 at optimization by adding more neurons to the second, third, and fourth hidden layers)
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Add fifth hidden layer (attempt 3 at optimization by adding another hidden layer with more neurons)
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 20)                620       
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9941 (38.83 KB)
Trainable params: 9941 (38

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 11s 6ms/step - loss: 0.5692 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7313
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7346
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5466 - accuracy: 0.7353
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5466 - accuracy: 0.7349
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accur

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5641 - accuracy: 0.7291 - 532ms/epoch - 2ms/step
Loss: 0.5640909075737, Accuracy: 0.7290962338447571


In [17]:
# Export our model to HDF5 file
hdf5_path = '/content/drive/My Drive/Colab Notebooks/deep-learning-challenge/AlphabetSoupCharity_Optimization.h5'
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import os
print (os.getcwd())

from google.colab import drive
drive.mount('/content/drive')
nn.save(hdf5_path)

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Report on the Neural Network Model**

**Overview of the analysis:** The provided code implements a neural network using TensorFlow and Keras for a binary classification task, predicting charity funding success. It involves data loading, preprocessing, model definition, training, evaluation, and export to an HDF5 file, enabling future use or deployment without retraining. The analysis process encompasses preparing the data, defining and training the model, evaluating its performance, and exporting it for practical application..

**What variable(s) are the target(s) for your model?**

IS_SUCCESSFUL is the target in the model

**What variable(s) are the features for your model?**

APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS,
INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT

**What variable(s) should be removed from the input data because they are neither targets nor features?**

NAME and EIN

**How many neurons, layers, and activation functions did you select for your neural network model, and why?**

Originally, there were 2 hidden layers and 1 output layer. I selected 80 neurons and 30 neurons initially for the hidden layers. 'Relu' was the activation function. I decided to start with this because it seemed like a strong initial foundation for building the model and achieving a high accuracy.

**Were you able to achieve the target model performance?**

No, I did not achieve target model performance.

**What steps did you take in your attempts to increase model performance?**

First try: Added a third addtional layer with 30 neurons.

Second try: Added a fourth additional layer with 20 neurons.

Third Try: Added a fifth addtional layer with 10 neurons.

**Summary:**

The deep learning model achieved 72% accuracy in predicting charity funding success. To potentially improve performance, an ensemble learning approach, such as Random Forest or Gradient Boosting, could be explored alongside feature engineering techniques. Ensemble methods can leverage diverse models to better capture complex patterns, while feature engineering can provide additional discriminative information, potentially enhancing the model's ability to differentiate between successful and unsuccessful funding applicants, all within the existing code framework.